# Cella 1 - Importazioni:

In [ ]:
# Importazione delle librerie necessarie
import os
import re
import pandas as pd
import numpy as np
from dotenv import load_dotenv
from decimal import Decimal
from connessione_sql import get_mysql_connection

load_dotenv()

# Configurazione pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Cella 2 - Carica dati da raw_data in DataFrame:

In [ ]:
# Connessione al database
conn = get_mysql_connection()

# Carica tutti i dati da raw_data in un DataFrame
query = "SELECT * FROM raw_data"
df_raw = pd.read_sql(query, conn)

conn.close()

print(f"Dataset caricato: {df_raw.shape[0]} righe, {df_raw.shape[1]} colonne")
print("\n Prime 5 righe:")
df_raw.head()

# Cella 3 - Esplora il dataset:

In [ ]:
# Informazioni sul dataset
print("📋 Informazioni sul dataset:\n")
df_raw.info()

print("\n📊 Valori nulli per colonna:")
print(df_raw.isnull().sum())

print("\n📊 Distribuzione contesto:")
print(df_raw['contesto'].value_counts())

# Cella 4 - Funzione pulizia età:

In [ ]:
# Funzione per pulire e convertire l'età
def pulisci_eta(eta_raw):
    """
    Converte l'età da testo a intero.
    FILTRO: Solo età tra 19 e 30 anni
    """
    if pd.isna(eta_raw) or eta_raw == '':
        return None
    
    numeri = re.findall(r'\d+', str(eta_raw))
    
    if numeri:
        eta_int = int(numeri[0])
        if 19 <= eta_int <= 30:
            return eta_int
    return None


# Applica la funzione al DataFrame
df_raw['età_pulita'] = df_raw['età'].apply(pulisci_eta)

print("🧪 Test pulizia età:\n")
print(df_raw[['età', 'età_pulita']].head(10))

print(f"\n⚠️ Righe da escludere (età fuori range 19-30): {df_raw['età_pulita'].isna().sum()}")

# Cella 5 - Funzione pulizia anni esperienza:

In [ ]:
# Funzione SEMPLIFICATA e CORRETTA
def pulisci_anni_esperienza(exp_raw):
    """Converte anni esperienza in formato anni.mesi"""
    if pd.isna(exp_raw) or exp_raw == '':
        return None
    
    exp_str = str(exp_raw).lower().replace(',', '.')
    
    totale_anni = 0
    totale_mesi = 0
    
    # Caso 1: "X anni"
    anni_match = re.findall(r'(\d+)\s*ann[oi]', exp_str)
    for a in anni_match:
        totale_anni += int(a)
    
    # Caso 2: "Y mesi"
    mesi_match = re.findall(r'(\d+(?:\.\d+)?)\s*mes[ei]', exp_str)
    for m in mesi_match:
        mesi_val = float(m)
        if mesi_val < 1:
            totale_mesi += int(mesi_val * 10)  # 0.7 mesi → 7 mesi (interpreta come 0.X anni)
        else:
            totale_mesi += int(mesi_val)
    
    # Caso 3: "mezzo"
    if 'mezzo' in exp_str or 'mezza' in exp_str:
        totale_mesi += 6
    
    # Caso 4: Numeri decimali (se non ha trovato "anni" o "mesi")
    if totale_anni == 0 and totale_mesi == 0:
        # Trova TUTTI i numeri decimali e sommali
        decimali = re.findall(r'\d+\.\d+', exp_str)
        
        for dec in decimali:
            val = float(dec)
            anni_parte = int(val)
            mesi_parte = int(round((val - anni_parte) * 10))  # Parte decimale × 10
            
            totale_anni += anni_parte
            totale_mesi += mesi_parte
    
    # Caso 5: Solo interi
    if totale_anni == 0 and totale_mesi == 0:
        numeri = re.findall(r'\b(\d+)\b', exp_str)
        if numeri:
            totale_anni = int(numeri[0])
    
    # Normalizza
    if totale_mesi >= 12:
        totale_anni += totale_mesi // 12
        totale_mesi = totale_mesi % 12
    
    # Formato finale
    return float(f"{totale_anni}.{totale_mesi:02d}")  # :02d garantisce sempre 2 cifre


# Test
print("🧪 Test:\n")
tests = ['1,1', '0,8', '9,5', '0,7 mesi', '2.5 lavoro di cui 0.5 studio']
for t in tests:
    print(f"'{t}' → {pulisci_anni_esperienza(t)}")

# Applica al DataFrame
df_raw['anni_esperienza_pulita'] = df_raw['anni_esperienza'].apply(pulisci_anni_esperienza)

print("\n📊 Test su dati reali:\n")
print(df_raw[['anni_esperienza', 'anni_esperienza_pulita']].head(40))

Cella 6 - Pulizia incoerenze contesto:

In [ ]:
# Funzione per pulire incoerenze tra contesto e risposte
def pulisci_incoerenze_items(row):
    """
    Pulisce incoerenze tra contesto e Items.
    - Se "Di studio" → Item_16-20 (lavoro) = NULL
    - Se "Di lavoro" → Item_11-15 (studio) = NULL
    - Se "Entrambi" → tutto OK
    """
    contesto = str(row['contesto']).lower()
    
    # Se contesto è "Di studio" → cancella Item_16-20
    if 'studio' in contesto and 'entrambi' not in contesto and 'lavoro' not in contesto:
        for i in range(16, 21):  # Item_16 a Item_20
            row[f'Item_{i}'] = None
    
    # Se contesto è "Di lavoro" → cancella Item_11-15
    elif 'lavoro' in contesto and 'entrambi' not in contesto and 'studio' not in contesto:
        for i in range(11, 16):  # Item_11 a Item_15
            row[f'Item_{i}'] = None
    
    return row


# Analisi PRIMA della pulizia
print("📊 PRIMA della pulizia incoerenze:\n")
for idx, row in df_raw.head(10).iterrows():
    contesto = row['contesto']
    studio_count = sum(1 for i in range(11, 16) if pd.notna(row[f'Item_{i}']))
    lavoro_count = sum(1 for i in range(16, 21) if pd.notna(row[f'Item_{i}']))
    print(f"ID {row['id']} | Contesto: {contesto:15s} | Studio: {studio_count}/5 | Lavoro: {lavoro_count}/5")

# Applica la pulizia
df_raw = df_raw.apply(pulisci_incoerenze_items, axis=1)

print("\n📊 DOPO la pulizia incoerenze:\n")
for idx, row in df_raw.head(10).iterrows():
    contesto = row['contesto']
    studio_count = sum(1 for i in range(11, 16) if pd.notna(row[f'Item_{i}']))
    lavoro_count = sum(1 for i in range(16, 21) if pd.notna(row[f'Item_{i}']))
    print(f"ID {row['id']} | Contesto: {contesto:15s} | Studio: {studio_count}/5 | Lavoro: {lavoro_count}/5")

In [ ]:
df_raw.head(20)

# Cella 7 - Filtra per età e crea DataFrame pulito:

In [ ]:
# Filtra solo righe con età valida (19-30)
df_clean = df_raw[df_raw['età_pulita'].notna()].copy()

# Sostituisci le colonne pulite
df_clean['età'] = df_clean['età_pulita'].astype(int)
df_clean['anni_esperienza'] = df_clean['anni_esperienza_pulita']

# Rimuovi colonne temporanee
df_clean = df_clean.drop(columns=['età_pulita', 'anni_esperienza_pulita'])

print(f"✅ Dati puliti:")
print(f"   - Righe prima: {len(df_raw)}")
print(f"   - Righe dopo (età 19-30): {len(df_clean)}")
print(f"   - Righe escluse: {len(df_raw) - len(df_clean)}")

print("\n📊 Statistiche età dopo pulizia:")
print(df_clean['età'].describe())

print("\n🔍 Prime 5 righe del DataFrame pulito:")
df_clean.head()